In [1]:
import pandas as pd
from conllu import parse

# Ввод

In [2]:
name = 'pisat_Dat'

object_form = {'Case': 'Dat', 'Animacy': 'Anim'}
verb = 'писать'
preposition = 'к'
if_preposition = True

# Данные и основные переменные

In [3]:
def create_path_and_file_name(name):
    path = '../data/{}.xlsx'.format(name)
    text_file_name = 'sents_{}.txt'.format(name)
    conllu_file_name = 'parsed_sents_{}.conllu'.format(name)
    file_name = [path, text_file_name, conllu_file_name]
    return file_name

In [4]:
names = create_path_and_file_name(name)

In [5]:
def read_preprocess_data(names):
    dataframe = pd.read_excel(names[0])
    dataframe.drop(columns=['Reversed left context', 'Reversed center', 'Title', 'Author', 'Birthday', 'Header', 
                            'Publ_year', 'Medium', 'Ambiguity', 'Publication', 'Sphere', 'Type', 'Topic'], inplace = True)
    return dataframe

In [6]:
df = read_preprocess_data(names)

In [7]:
df

,Left context,Center,Punct,Right context,Created,Full context
0,"Кунцево"", о котором я уже",писал,,"в статье "" Мастер к мастеру, лоскуток",2004,"Это делают мастерицы московского клуба "" Красн..."
1,"припомнила вдруг, что выпускное сочинение",писала,,по « Матери» Горького на тему « Родные,2004,"Я пока в соседнем своём топе думы думала, прип..."
2,"этой жизни, а Вы сами",пишете,",",что девочке не хватает внимания.,2004,Но ведь в этом ребенкином возрасте понять всё ...
3,Александру Волошину пришлось,писать,,патриарху Алексию и просить обеспечить невмеш...,2003,Александру Волошину пришлось писать патриарху ...
4,"Что-то дикое есть в сознании, ―",писал,,он летом 1955-го Борису Константиновичу Зайцев...,2003,"Что-то дикое есть в сознании, ― писал он летом..."
5,после переезда в Йер он,писал,,"в Нью-Йорк прозаику Роману Борисовичу Гулю,",2003,Примерно через месяц после переезда в Йер он п...
6,"нашей адской жизни последних лет"", ―",пишет,,он Гулю.,2003,"Если не заниматься высокими делами, то всё-так..."
7,"но не особенно успешно лечусь"", ―",писал,,он Гулю.,2003,"Я бывший пьяница, от последствий чего упорно, ..."
8,тосковал по Парижу и саркастически,писал,,"Лидии Червинской: "" В ни один из",2003,Иванов тосковал по Парижу и саркастически писа...
9,"вот уже нескольких последних лет, ―",пишут,,Путину председатель Российского профсоюза раб...,2003,Рыбаки России крайне обеспокоены и возмущены т...


In [8]:
df["Created"].head()

0    2004
1    2004
2    2004
3    2003
4    2003
Name: Created, dtype: object

In [9]:
from finding_my_triplets import get_all_triples, read_conllu, get_standart_date, get_indexes

In [10]:
sentences2 = read_conllu('parsed_sents_pisat_Dat.conllu')
object_form = {'Case': 'Dat'} #, 'Animacy': 'Anim'}
verb = 'писать'
preposition = False
if_preposition = True
triplets2 = get_all_triples(sentences2, verb, object_form, preposition, if_preposition)

In [11]:
ids = get_indexes(triplets2) # индексы предложений, которые подходят и для которых узнаём дату

In [14]:
result = {
    (1700, 1749): [],
    (1750, 1799): [],
    (1800, 1849): [],
    (1850, 1899): [],
    (1900, 1949): [],
    (1950, 1999): [],
    (2000, 2049): []
}

for ind in ids:
    date = df.iloc[ind].Created # получение по индексу даты создания из DataFrame
    date = get_standart_date(date)
    date = int(date)
    for start_year, end_year in result:
        if date >= start_year and date <= end_year:
            result[(start_year, end_year)].append(ind)

In [15]:
result

{(1700, 1749): [],
 (1750, 1799): [115],
 (1800, 1849): [114],
 (1850, 1899): [109, 110, 111],
 (1900, 1949): [82,
  83,
  85,
  87,
  90,
  92,
  93,
  95,
  96,
  97,
  98,
  99,
  101,
  102,
  104,
  105,
  106,
  107,
  108],
 (1950, 1999): [19,
  20,
  22,
  24,
  27,
  28,
  30,
  31,
  32,
  35,
  41,
  42,
  50,
  54,
  57,
  58,
  60,
  62,
  64,
  65,
  68,
  73,
  74,
  75,
  76,
  77],
 (2000, 2049): [0,
  3,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  15,
  17,
  118,
  120,
  121,
  124,
  126,
  134,
  139,
  153,
  157,
  158,
  161,
  165,
  173,
  178,
  179,
  182,
  186,
  190,
  192,
  193,
  194,
  195,
  200,
  203,
  205,
  215,
  220,
  222,
  226,
  228,
  229,
  232,
  237,
  238,
  239,
  240,
  244,
  249,
  251,
  252,
  261,
  262,
  266,
  267,
  271,
  279,
  284,
  285,
  287,
  294,
  296,
  301,
  302,
  303,
  314,
  318,
  334,
  335,
  336,
  337,
  338,
  339,
  340,
  342,
  344,
  345,
  347,
  355,
  357,
  360,
  361,
  371,
  372,
  380,
 

In [10]:
df['Full context'].iloc[0]

'Это делают мастерицы московского клуба " Красный сарафан", при фольклорном центре ТКС ( территориально-клубная система) " Кунцево", о котором я уже писал в статье " Мастер к мастеру, лоскуток к лоскутку"" НТ" N 6 за 2002 г.. [Народный костюм: архаика или современность? // «Народное творчество», 2004] [омонимия снята]'

In [12]:
# df['Created']

In [39]:
def create_text_file(dataframe, names):
    with open(names[1], 'w', encoding = "UTF-8") as f:
        for i in range(dataframe.shape[0]):
            sent = dataframe['Full context'][i]
            f.write(sent[:sent.find('[')-1] + '\n')

In [40]:
def read_conllu(names):
    with open(names[2], 'r', encoding = "UTF-8") as f:
        parsed_sents = f.read()
        sents = parse(parsed_sents)
        return sents

In [41]:
create_text_file(df, names)

## Ввести названия файлов для парсера

In [42]:
print(names[1], '\n', names[2], sep = '')

sents_pisat_Dat.txt
parsed_sents_pisat_Dat.conllu


In [44]:
!udpipe --tokenize --tag --parse\
    ../models/russian-syntagrus-ud-2.0-170801.udpipe sents_pisat_Dat.txt > parsed_sents_pisat_Dat.conllu

Loading UDPipe model: done.


In [39]:
sents = read_conllu(names)

In [101]:
sents[0][34]

OrderedDict([('id', 35),
             ('form', 'мастеру'),
             ('lemma', 'мастер'),
             ('upostag', 'NOUN'),
             ('xpostag', None),
             ('feats',
              OrderedDict([('Animacy', 'Inan'),
                           ('Case', 'Dat'),
                           ('Gender', 'Masc'),
                           ('Number', 'Sing')])),
             ('head', 29),
             ('deprel', 'obl'),
             ('deps', None),
             ('misc', OrderedDict([('SpaceAfter', 'No')]))])

In [74]:
find_triplet_in_sentence(sents[4], verb, object_form, preposition, if_preposition)

{'verb': 'писали'}

# Нахождение триплетов

In [41]:
def check_object(token, object_form):
    if 'Foreign' in token['feats'] and token['feats']['Foreign'] == 'Yes':
        return False
    for feature in object_form:
        if token['feats'][feature] != object_form[feature]:
            return False
    return True

In [42]:
def find_triplet_in_sentence(sent, verb, object_form, preposition=None, if_preposition=None):
    # TODO: подумать про случаи, когда несколько подходящих глаголов в предложении
    triplet = {}
    verb_id = None
    object_id = None
    for token in sent:
        
        # ищем id глагола, чтобы потом искать его зависимые
        if token['lemma'] == verb:
            verb_id = token['id']
            triplet['verb'] = token['form']
            
        # ищем зависимые (NB: в нашем случае зависимые всегда идут после глагола, поэтому в одном цикле)
        if token['head'] == verb_id:
            if token['upostag'] in ['NOUN', 'PROPN'] and check_object(token, object_form):
                object_id = token['id'] - 1
                break # мы нашли объект (TODO: bla bla)
                
    if preposition and object_id: # looking for a preposition, if there is
        if if_preposition:
            for token in sent:
                if token['head'] == object_id and token['form'] == preposition:
                    triplet['object'] = sent[object_id]['form']
                    triplet['preposition'] = preposition
        else:
            for token in sent: # looking for a preposition, if there isn't
                if token['head'] == object_id and token['form'] == preposition:
                    break
            else:
                triplet['object'] = sent[object_id]['form']
    return triplet

In [43]:
n = 0
print(sents[n])
find_triplet_in_sentence(sents[n], verb, object_form, preposition, if_preposition)

TokenList<Это, делают, мастерицы, московского, клуба, ", Красный, сарафан, ", ,, при, фольклорном, центре, ТКС, (, территориально, -, клубная, система, ), ", Кунцево, ", ,, о, котором, я, уже, писал, в, статье, ", Мастер, к, мастеру, ,, лоскуток, к, лоскутку, ", ", НТ, ", N, 6, за, 2002, г., .>


{'verb': 'писал'}

In [44]:
def get_all_triples(sentences, verb, object_form, preposition=None, if_preposition=None):
    triples = []
    for i, sent in enumerate(sentences):
        triplet = find_triplet_in_sentence(sent, verb, object_form, preposition, if_preposition)
        triplet['id'] = i
        triples.append(triplet)
    return triples

In [45]:
triples = get_all_triples(sents, verb, object_form, preposition, if_preposition)

In [61]:
sents[9]

TokenList<В, последнее, время, я, занимался, тем, ,, что, писал, предисловие, к, тому, Ильи, Кормильцева, и, к, «, Капиталу», Маркса, ,, переизданному, в, честь, 150, -, летия, со, дня, его, публикации, в, Гамбурге, .>

In [51]:
triples

[{'verb': 'писал', 'id': 0},
 {'verb': 'писал', 'id': 1},
 {'verb': 'пишет', 'id': 2},
 {'verb': 'писал', 'id': 3},
 {'verb': 'писали', 'id': 4},
 {'verb': 'писать', 'id': 5},
 {'verb': 'писал', 'id': 6},
 {'verb': 'писал', 'id': 7},
 {'verb': 'пишу', 'id': 8},
 {'verb': 'писал', 'id': 9},
 {'verb': 'писать', 'id': 10},
 {'verb': 'писал', 'id': 11},
 {'verb': 'писал', 'id': 12},
 {'verb': 'писал', 'id': 13},
 {'verb': 'пишут', 'id': 14},
 {'id': 15},
 {'id': 16},
 {'verb': 'писал', 'id': 17},
 {'verb': 'писали', 'id': 18},
 {'verb': 'писали', 'id': 19},
 {'verb': 'писала', 'id': 20},
 {'id': 21},
 {'verb': 'пишет', 'id': 22},
 {'verb': 'пишет', 'id': 23},
 {'verb': 'писал', 'id': 24},
 {'id': 25},
 {'id': 26},
 {'verb': 'писала', 'id': 27},
 {'verb': 'писать', 'id': 28},
 {'verb': 'писать', 'id': 29},
 {'verb': 'писал', 'id': 30},
 {'verb': 'писал', 'id': 31},
 {'verb': 'писала', 'id': 32},
 {'verb': 'писала', 'id': 33},
 {'verb': 'пишет', 'id': 34},
 {'verb': 'пишу', 'id': 35},
 {'id'

In [75]:
sents[527]

TokenList<Согласен, ,, что, писать, к, нему, Егору, Егорычу, неловко, ,, ехать, самому, тем, паче, ,, но, не, выкинуть, ли, такую, штуку, :, не, съездить, ли, мне, к, Валерьяну, Николаичу, и, по, душе, поговорить, с, ним, ?>

In [49]:
def count_triples(triples):
    count = 0
    for triple in triples:
        if triple.get('object') != None:
            count+=1
    return count

In [50]:
count_triples(triples)

4

In [47]:
def count_triples(triples):
    count = 0
    for index, triple in enumerate(triples):
        if triple.get('object') != None:
            count+=1
#             dataframe.loc[index, {triple}] = triple['verb']
    return count

In [48]:
count_triples(triples)

4

## Пример
касаться до gen vs. касаться gen (от касаться зависит NOUN/PROPN + case-Gen (?) и не зависит до)